In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_shape = train_images.shape
test_shape = test_images.shape

y_shape = len(set(train_labels))

train_images_reshaped = train_images.reshape(train_shape[0],train_shape[1]*train_shape[2])/255.0
test_images_reshaped = test_images.reshape(test_shape[0],train_images_reshaped.shape[1])/255.0

In [3]:
hidden_sizes = [1000,1000,500,200]

In [4]:
tf.reset_default_graph()

with tf.device("/GPU:0"):
    x_placeholder = tf.placeholder(tf.float32,shape = (None,train_shape[1]*train_shape[2]))
    y_placeholder = tf.placeholder(tf.int64,shape=(None))
    output_layer_weights =  tf.Variable( tf.truncated_normal([hidden_sizes[-1],y_shape]))


    weights_matrices = []
    layer_outputs = []

    layer_outputs.append(x_placeholder)

    for layer in range(len(hidden_sizes)):
        if layer == 0:

            prev_units = int(x_placeholder.shape[1]) 
        else:
            prev_units = int(weights_matrices[layer-1].shape[1])

        layer_units = hidden_sizes[layer]

        print(layer+1, (prev_units,layer_units))
        weights_matrix_l = tf.Variable( tf.truncated_normal ([prev_units,layer_units]))
        layer_output =  tf.nn.relu( tf.matmul(layer_outputs[layer],weights_matrix_l))

        weights_matrices.append(weights_matrix_l)
        layer_outputs.append(layer_output)

    last_layer_output =    tf.matmul(  layer_output , output_layer_weights)

    print(layer+2,  (hidden_sizes[-1],y_shape))

    loss =  tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y_placeholder,logits = last_layer_output))
    optimizer_step = tf.train.AdamOptimizer(0.01).minimize(loss)

    last_layer_hard_output = tf.argmax(last_layer_output,axis=1)
    accuracy = tf.reduce_mean( tf.cast( tf.equal(last_layer_hard_output,y_placeholder),tf.float32))


    init = tf.global_variables_initializer()

1 (784, 1000)
2 (1000, 1000)
3 (1000, 500)
4 (500, 200)
5 (200, 10)


In [5]:
EPOCHS = 100
PRINT_EVERY = 10

In [13]:
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.InteractiveSession(config=config)

sess.run(init)
feed_dict = {x_placeholder:train_images_reshaped,y_placeholder:train_labels}

for epoch in range(1,EPOCHS + 1):
    _,epoch_loss,epoch_output,epoch_hard_output,epoch_acc  = sess.run([optimizer_step,loss,last_layer_output,last_layer_hard_output,accuracy],feed_dict)
    
    if epoch % PRINT_EVERY == 0:
        print("Epoch :{} loss:{}  epoch acc:{}".format(epoch,epoch_loss,epoch_acc))
        

trained_weights =  sess.run([layer  for layer in  weights_matrices],feed_dict=feed_dict)


Epoch :10 loss:63435.0078125  epoch acc:0.6527166962623596
Epoch :20 loss:19291.916015625  epoch acc:0.803433358669281
Epoch :30 loss:9944.955078125  epoch acc:0.8721333146095276
Epoch :40 loss:5794.23876953125  epoch acc:0.9075999855995178
Epoch :50 loss:3983.037841796875  epoch acc:0.921833336353302
Epoch :60 loss:2991.695556640625  epoch acc:0.9301999807357788
Epoch :70 loss:2316.428466796875  epoch acc:0.9383166432380676
Epoch :80 loss:1823.1290283203125  epoch acc:0.9466500282287598
Epoch :90 loss:1445.6376953125  epoch acc:0.9542666673660278
Epoch :100 loss:1146.9637451171875  epoch acc:0.9608666896820068


In [12]:
test_feed_dict = {x_placeholder:test_images_reshaped,y_placeholder:test_labels}
test_acc = sess.run(accuracy,feed_dict=test_feed_dict)

print("Test acc",test_acc)


Test acc 0.9309


In [15]:
trained_weights


[array([[-4.03605662e-02, -1.01700410e-01, -3.49279009e-02, ...,
          1.72350514e+00,  9.12831247e-01,  1.31312096e+00],
        [-9.88243461e-01,  8.41427505e-01,  1.11209893e+00, ...,
          8.79794359e-01,  2.60582268e-02, -1.85387456e+00],
        [ 9.35282648e-01,  5.62196612e-01,  2.00398508e-02, ...,
         -7.16317445e-02, -1.85424149e-01,  1.15086126e+00],
        ...,
        [ 1.62240887e+00, -2.48845220e-01,  1.47115008e-03, ...,
          3.02017450e-01,  1.76415648e-02,  3.59472036e-01],
        [-8.64456177e-01,  1.99837232e+00, -1.55694389e+00, ...,
         -1.31466925e+00,  3.73141915e-01, -1.20031215e-01],
        [-1.84592068e-01, -6.74613476e-01,  2.33404532e-01, ...,
         -5.63059114e-02, -1.06725788e+00, -1.29767644e+00]], dtype=float32),
 array([[-0.74925524,  0.38654003,  1.8252498 , ...,  0.40536758,
         -0.71222305,  1.5640484 ],
        [-0.50121474,  1.141338  , -0.47750545, ...,  0.54105234,
          1.0982342 , -0.8031056 ],
        [ 